In [1]:
from fastai.tabular import *

# Tabular data

In [6]:
path = untar_data(URLs.ADULT_SAMPLE) #fastai datset (URLs)
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [4]:
dependent_var = 'salary'
categorical_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
continuous_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize] #processors are in tab data like transfroms in vision but they just happen once ahead of time
#FillMissing: Look for missing values and deal with them some way.
#Categorify: Find categorical variables and turn them into Pandas categories
#Normalize : Do a normalization ahead of time which is to take continuous variables and subtract their mean and divide by their standard deviation so they are zero-one variables.

In [5]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=categorical_names, cont_names=continuous_names)

In [6]:
data = (TabularList.from_df(df, path=path, cat_names=categorical_names, cont_names=continuous_names, procs=procs)
                            .split_by_idx(list(range(800,1000)))
                            .label_from_df(cols=dependent_var)
                            .add_test(test, label=0)
                            .databunch())

In [9]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,11th,Divorced,Sales,Unmarried,White,False,-1.4357,0.0761,-1.2046,<50k
Private,Bachelors,Divorced,Sales,Not-in-family,White,False,0.1769,-1.2872,1.1422,<50k
Private,Some-college,Never-married,Handlers-cleaners,Own-child,White,False,-1.2158,0.8695,-0.0312,<50k
Self-emp-not-inc,Masters,Divorced,Exec-managerial,Not-in-family,White,False,0.5434,-1.0316,1.5334,<50k
Self-emp-inc,HS-grad,Never-married,Sales,Other-relative,White,False,0.9098,-0.5707,-0.4224,<50k
Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,Black,False,0.6166,0.4068,-0.4224,<50k
Private,Assoc-voc,Married-civ-spouse,Machine-op-inspct,Husband,White,False,0.6166,0.0479,0.3599,>=50k
Private,HS-grad,Separated,Other-service,Not-in-family,White,False,-0.5561,1.3368,-0.4224,<50k
State-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,-0.4095,1.3525,-0.4224,>=50k
Private,9th,Separated,Machine-op-inspct,Unmarried,White,False,-0.3362,-0.1136,-1.9869,<50k


In [10]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [11]:
learn.fit(1,1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.364372,0.407050,0.800000,00:03


## Inference

In [12]:
row = df.iloc[0]

In [13]:
learn.predict(row)

(Category >=50k, tensor(1), tensor([0.4919, 0.5081]))

In [14]:
df.iloc[0]

age                                49
workclass                     Private
fnlwgt                         101320
education                  Assoc-acdm
education-num                      12
marital-status     Married-civ-spouse
occupation                        NaN
relationship                     Wife
race                            White
sex                            Female
capital-gain                        0
capital-loss                     1902
hours-per-week                     40
native-country          United-States
salary                          >=50k
Name: 0, dtype: object

In [15]:
learn.predict(df.iloc[25])

(Category >=50k, tensor(1), tensor([0.2231, 0.7769]))

In [16]:
df.iloc[25]

age                                55
workclass                Self-emp-inc
fnlwgt                         222615
education                     Masters
education-num                      14
marital-status     Married-civ-spouse
occupation            Exec-managerial
relationship                  Husband
race                            White
sex                              Male
capital-gain                        0
capital-loss                        0
hours-per-week                     60
native-country          United-States
salary                           <50k
Name: 25, dtype: object